# Notebook para realizar gridsearch no GSGP

A implementação original não tinha funções de transformação iguais às utilizadas no ITEA, e utilizava o MAE como função de custo. Marquei no código com os seguintes comentários:

* MODIFICAÇÃO: comentário que precede uma modificação que eu fiz no código original
* PONTO DE INTERESSE: comentário que precede um ponto do código o qual pode ser interessante ter uma referência, pois modificações futuras podem precisar que o ponto seja levado em consideração
* OBSERVAÇÃO: observações gerais que encontrei no código (bugs, comportamentos estranhos, detalhes).

## Formato de entreada:

O programa tem um formato um pouco chato de arquivos, e é necessário converter os datasets que utilizamos nos outros programas para uma forma adequada. Pela documentação:

> Input files are .txt files where values are separated by a TAB. The first two rows of each file represent the number of independent variables of the problem and the number of instances in the dataset. Each row represents an instance, while each column contains the values of a variable. The last column contains the target values. The library comes with two example input files (training and test files).

> NOTE: in test mode the test file must not have the number of instances in the dataset and also the target column must be removed.

A função que gera as bases de dados de forma adequada irá criar tudo dentro da pasta ../datasets_GSGP

## Novas funções de transformação

Vou adicionar as funções:
{id,sin, cos,tanh, sqrt|.|, log, exp}

## Gridsearch

Como a implementação é em C++ mas python tem mais praticidade para automatizar os testes, esse notebook implementa funções que manipulam os arquivos relacionados ao programa, executam diferentes configurações e salvam os resultados obtidos.


## Expressão GSGP:

a reconstrução leva em conta indivíduos de todas as gerações, sendo inviável reconstruir por conta do crescimento exponencial do tamanho da árvore. Não terá utilização prática obter a expressão final, portanto não me dediquei a escrever um código que reconstruísse ela (apesar de ter a descrição no artigo).

In [1]:
import os.path   as path
import os
import glob
import pandas as pd
import numpy as np
from itertools import product
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import KFold

# Diretório atual
cur_folder = os.getcwd()

original_datasets_folder = '../datasets'
adapted_datasets_folder  = '../datasets_GSGP'

# compilar o código fonte
! cd "{cur_folder}/"
! g++ GP.cc -o GP.exe

print('Done')

Done


## Gerando as bases de dados

Só executar se a pasta datasets_GSGP estiver vazia

In [2]:
# Criar folder dcom bases de dados ajustadas
if not os.path.exists(f'{cur_folder}/{adapted_datasets_folder}'):
    os.makedirs(f'{cur_folder}/{adapted_datasets_folder}')
    
# Executar para cada base de dados
for dataset in glob.glob(f'{cur_folder}/{original_datasets_folder}/*.dat'):

    # Pegando o nome do arquivo
    file_name = dataset.replace(f'{cur_folder}/{original_datasets_folder}/', '').replace('.dat', '')
    
    print(f'{cur_folder}/{adapted_datasets_folder}/{file_name}.txt')
    
    # Vou abrir com o pandas e salvar apenas trocando o separador
    df = pd.read_csv(dataset, header=None, sep=',')
    
    # Pegando informações das dimensões dos dados
    nrow, nvar = df.shape
    nvar = nvar - 1
    
    # Salvando com separador adequado os dados de treino e teste para a EVOLUÇÂO
    df.to_csv(
        f'{cur_folder}/{adapted_datasets_folder}/{file_name}.txt',
        sep='\t', index=None, header=None, float_format ='%1.9f'
    )
    
    # Colocando no início do arquivo as dimensões da base, criando o treino e teste
    with open(f'{cur_folder}/{adapted_datasets_folder}/{file_name}.txt', 'r+') as f:
        content = f.read()
        f.seek(0, 0)
        f.write(f'{nvar}\n{nrow}\n' +  content[:-1])
        
    # Se for partição de treino, queremos a divisão em 5-fold para a validação cruzada
    # Então vamos gerar 5 folds e salvar como train-i-train-j, e train-i-validation-j
    if 'train' in file_name:
        # Para cada um vamos repetir o que fizemos lá em cima
        for i, (train_index, test_index) in enumerate(KFold(n_splits=5).split(df.iloc[:, :-1])):
            # Salvando com separador adequado os dados de treino e teste para a EVOLUÇÂO
            df.iloc[train_index, :].to_csv(
                f'{cur_folder}/{adapted_datasets_folder}/{file_name}-train-{i}.txt',
                sep='\t', index=None, header=None, float_format ='%1.9f'
            )

            # Colocando no início do arquivo as dimensões da base, criando o treino e teste
            with open(f'{cur_folder}/{adapted_datasets_folder}/{file_name}-train-{i}.txt', 'r+') as f:
                content = f.read()
                f.seek(0, 0)
                f.write(f'{nvar}\n{len(train_index)}\n' +  content[:-1])
                
            df.iloc[test_index, :].to_csv(
                f'{cur_folder}/{adapted_datasets_folder}/{file_name}-validation-{i}.txt',
                sep='\t', index=None, header=None, float_format ='%1.9f'
            )

            # Colocando no início do arquivo as dimensões da base, criando o treino e teste
            with open(f'{cur_folder}/{adapted_datasets_folder}/{file_name}-validation-{i}.txt', 'r+') as f:
                content = f.read()
                f.seek(0, 0)
                f.write(f'{nvar}\n{len(test_index)}\n' +  content[:-1])

/mnt/c/Users/Guilherme Aldeia/Documents/github/experiments-ITEA-paper/src/../datasets_GSGP/airfoil-test-0.txt
/mnt/c/Users/Guilherme Aldeia/Documents/github/experiments-ITEA-paper/src/../datasets_GSGP/airfoil-test-1.txt
/mnt/c/Users/Guilherme Aldeia/Documents/github/experiments-ITEA-paper/src/../datasets_GSGP/airfoil-test-2.txt
/mnt/c/Users/Guilherme Aldeia/Documents/github/experiments-ITEA-paper/src/../datasets_GSGP/airfoil-test-3.txt
/mnt/c/Users/Guilherme Aldeia/Documents/github/experiments-ITEA-paper/src/../datasets_GSGP/airfoil-test-4.txt
/mnt/c/Users/Guilherme Aldeia/Documents/github/experiments-ITEA-paper/src/../datasets_GSGP/airfoil-train-0.txt
/mnt/c/Users/Guilherme Aldeia/Documents/github/experiments-ITEA-paper/src/../datasets_GSGP/airfoil-train-1.txt
/mnt/c/Users/Guilherme Aldeia/Documents/github/experiments-ITEA-paper/src/../datasets_GSGP/airfoil-train-2.txt
/mnt/c/Users/Guilherme Aldeia/Documents/github/experiments-ITEA-paper/src/../datasets_GSGP/airfoil-train-3.txt
/mnt/c

/mnt/c/Users/Guilherme Aldeia/Documents/github/experiments-ITEA-paper/src/../datasets_GSGP/wineRed-train-1.txt
/mnt/c/Users/Guilherme Aldeia/Documents/github/experiments-ITEA-paper/src/../datasets_GSGP/wineRed-train-2.txt
/mnt/c/Users/Guilherme Aldeia/Documents/github/experiments-ITEA-paper/src/../datasets_GSGP/wineRed-train-3.txt
/mnt/c/Users/Guilherme Aldeia/Documents/github/experiments-ITEA-paper/src/../datasets_GSGP/wineRed-train-4.txt
/mnt/c/Users/Guilherme Aldeia/Documents/github/experiments-ITEA-paper/src/../datasets_GSGP/wineWhite-test-0.txt
/mnt/c/Users/Guilherme Aldeia/Documents/github/experiments-ITEA-paper/src/../datasets_GSGP/wineWhite-test-1.txt
/mnt/c/Users/Guilherme Aldeia/Documents/github/experiments-ITEA-paper/src/../datasets_GSGP/wineWhite-test-2.txt
/mnt/c/Users/Guilherme Aldeia/Documents/github/experiments-ITEA-paper/src/../datasets_GSGP/wineWhite-test-3.txt
/mnt/c/Users/Guilherme Aldeia/Documents/github/experiments-ITEA-paper/src/../datasets_GSGP/wineWhite-test-4.

## Funções que realizam o gridsearch

In [4]:
# Função para atualizar o arquivo de configuração
# de acordo com um dicionario de configuração recebido
def update_config(conf, pred=False):
    # Pred determina se é para gerar a configuração com ou sem
    # o uso dos resultados da evolução
    
    data = []
    with open(f'{cur_folder}/configuration.ini', 'r+') as f:
        data = f.read().splitlines()

    for line_id, line in enumerate(data):
        param = line.split(' ')[0]
        value = line.split(' ')[-1]
        
        if param == 'expression_file': #caso seja para executar teste
            data[line_id] = line.replace(str(value), f'{1 if pred else 0}\n')
        elif param == 'USE_TEST_SET': # caso seja para executar teste
            data[line_id] = line.replace(str(value), f'{1 if pred else 0}\n')
        else: # copio os outros valores
            data[line_id] = line.replace(str(value), f'{conf[param]}\n')

    with open(f'{cur_folder}/configuration.ini', 'w') as f:
        f.writelines( data )
        

# Função que recebe partições de treino e validação (5-fold), e uma
# lista de configurações, e determina a melhor configuração. 
# Retorna o RMSE médio da melhor configuração, um dicionário da melhor
# configuração, e o id da melhor configuração.
# A configuração encontrada será utilizada em todas as repetições de uma divisão treino-teste
def gridsearch(dataset_train_cv, dataset_validation_cv, confs):
    
    # Executando para a primeira configuração
    
    update_config(confs[0], pred=False)
    
    print(f'Testando configuração 0/{len(confs)}')
    RMSE_cv = []
    for train_cv, validation_cv in zip(dataset_train_cv, dataset_validation_cv):
        ! ./GP.exe -train_file {train_cv} -test_file {validation_cv}
        cv_test = pd.read_csv(f'{cur_folder}/fitnesstest.txt', header=None)
        RMSE_cv.append(cv_test.iloc[-1, 0])
        
    # (rmse_cv, configuração, indice da configuração)
    best_conf = (np.mean(RMSE_cv), confs[0], 0)
    
    for i, conf in enumerate(confs[1:]):
        update_config(conf, pred=False)
        
        print(f'Testando configuração {i+1}/{len(confs)}')
        RMSE_cv = []
        for train_cv, validation_cv in zip(dataset_train_cv, dataset_validation_cv):
            ! ./GP.exe -train_file {train_cv} -test_file {validation_cv}
            
            cv_test = pd.read_csv(f'{cur_folder}/fitnesstest.txt', header=None)
            RMSE_cv.append(cv_test.iloc[-1, 0])

        if np.mean(RMSE_cv) < best_conf[0]:
            best_conf = (np.mean(RMSE_cv), conf,  i+1)
            
    return best_conf

        
# Função que executa uma configuração específica para uma divisão
# de treino e teste.
def run(dataset_train, dataset_test, conf=None):
    # Executando para os dados fornecidos
    
    # Modificando o arquivo de configuração para ficar igual à configuração
    # sendo utilizada
    update_config(conf, pred=False)
    
    ! ./GP.exe -train_file {dataset_train} -test_file {dataset_test}

    # Pegando o melhor fitness no treino e no teste (
    # esses arquivos guardam o melhor em cada geração, então a última linha
    # é o melhor)
    
    fitness_train    = pd.read_csv(f'{cur_folder}/fitnesstrain.txt', header=None)
    fitness_test     = pd.read_csv(f'{cur_folder}/fitnesstest.txt', header=None)
    
    return (
        fitness_train.iloc[-1, 0],
        fitness_test.iloc[-1, 0],
    )

## Considerações para escolha dos valores dos parâmetros:

Considerações com base na Q2 do revisor 1, e sugestões para cada ponto mencionado:

1. **"First of all, there is, in principle, no reason why should not use the same set of primitive functions as the other studied GP frameworks":**
    * Feito - funções de transformação utilizadas no ITEA foram implementadas no GSGP. Foi utilizado um framework em C/C++, e o código foi adaptado para utilizar o RMSE como função de fitness.
    
2. **"GSGP works better for smaller populations and larger number of generations":**
    * Sugestão: **population_size** e **max_number_generations** de gerações variando em [100, 250, 500]. Não pensei em ter mais pois tem um parâmetro **random_tree** que "aumenta" o tamanho da população
    
3. **"Tt is not clear if geometric semantic crossover is used or not; given that no crossover rate is given":**
    * Sugestão: **p_crossover** variando [0.05, 0.1, 0.25, 0.5]
    
4. **"The mutation rates that have been used look extremely small; this seems to strongly limit the exploitation power of GSGP; in GSGP, mutation rate should be high":**
    * Sugestão: mesmo do anterior: **p_mutation** variando [0.05, 0.1, 0.25, 0.5]
    
5. **"No mutation _step_ is discussed, and the mutation step is a crucial parameter for GSGP; this parameter should be attentively fine tuned (also, spread probability and \alpha, mentioned at page 13, have never been introduced):"**
    * Sugestão: **mutation_step** variando de [0.05, 0.1, 0.25, 0.5, 1.0]. Spread probability e alpha não existem nessa implementação (o primeiro soa como algo relacionado a crossover, e o segundo a mutação).
    
```python
gridsearch_configurations = {
    'population_size'        # Tamanho da população
    'max_number_generations' # Número de gerações (não tem stop criteria no código)
    'init_type'              # full growth, ramped half-half, ...
    'p_crossover'            # Probabilidade de crossover
    'p_mutation'             # Probabilidade de mutação (deve somar no máximo 1 com a anterior)
    'max_depth_creation'     # Maior profundidade. Como indivíduos são combinados, tanto faz?
    'tournament_size'        # Número de "confrontos" a cada torneio 
    'zero_depth'             # ACHO que é o tamanho mínimo
    'mutation_step'          # Passo de mutação, sorteado aleatóriamente no código (essa conf não é utilizada)
    'num_random_constants'   # Permitir ou não constantes aleatórias (gera segmentation fault)
    'min_random_constant'    # Menor valor de constante aleatória permitido
    'max_random_constant'    # Maior valor de constante aleatória permitido
    'minimization_problem'   # minimizar ou maximizar o fitness (minimizar, no caso)
    'random_tree'            # Número de árvores aleatórias além da população inicial
    
    # As de baixo ficam reservadas para o script controlar. Elas são relacionadas a
    # executar a evolução ou usar um resultado encontrado
    
    #'expression_file'        : 0,
    #'USE_TEST_SET'           : 0
}
```

**A execução do bloco de código abaixo irá gerar uma lista confs com todas as configurações de acordo com o que for especificado no dicionário gridsearch_configurations**

Uma tabela apresentando as configurações será apresentada, e um arquivo configurations.csv irá guardar a relação de configurações com os números utilizados. As configurações são enumeradas para deixar o arquivo de resultados mais breve (além do mais, acho que não serão utilizadas futuramente, mas uma referência para elas é feita para evitar problemas)

In [7]:
# Criando diferentes configurações:

# Listas com valores diferentes para testar no gridsearch.
# são permitidos: valores únicos, listas (será feito um produto cartesiano
# entre as listas), e funções lambda (que serão calculadas para cada
# configuração do produto cartesiano)
gridsearch_configurations = {
    'population_size'        : [100, 250, 500],
    'max_number_generations' : lambda conf:  100000//conf['population_size'],
    'init_type'              : 2,
    'p_crossover'            : [0, 0.2, 0.5],
    'p_mutation'             : lambda conf: 1 - conf['p_crossover'],
    'max_depth_creation'     : 5,
    'tournament_size'        : 4,
    'zero_depth'             : 0,
    'mutation_step'          : 1.0, 
    'num_random_constants'   : 0,
    'min_random_constant'    : -100,
    'max_random_constant'    : 100,
    'minimization_problem'   : 1,
    'random_tree'            : 500
    
    # As de baixo ficam reservadas para o script controlar. Elas são relacionadas a
    # executar a evolução ou usar um resultado encontrado
    
    #'expression_file'        : 0,
    #'USE_TEST_SET'           : 0
}

varying = []

keys, values = [], []
for k,v in gridsearch_configurations.items():
    if isinstance(v, list): # parâmetro é uma lista de valores possíveis
        values.append(v)
        varying.append(k)
    elif (isinstance(v, int) or isinstance(v, float)): # parâmetro é um valor único (fixado). Vamos colocar em uma lista para usar o product
        values.append([v])
    elif callable(v): # É uma função, fica pra depois
        continue
    else:
        raise Exception('o que é isso?')
    keys.append(k)
        
# Gerando todas as configurações que serão testadas
confs = [dict(zip(keys,items)) for items in product(*values)]

# Agora vamos pegar as funções lambda e aplicar
for conf in confs:
    for k,v in gridsearch_configurations.items():
        if callable(v): # É uma função, fica pra depois
            conf[k] = v(conf)
            varying.append(k)
                
# Verificar se temos em todas as confs o mesmo número
# que deveriamos ter do dicionário de valores
for conf in confs:
    if set(conf.keys()) != set(gridsearch_configurations.keys()):
        raise Exception(f'Parâmetros de busca e da configuração específica não batem. Configuração:\n{conf}')

# Criando um dataframe para enumerar e visualizar melhor as configurações
confs_df = pd.DataFrame(confs, index=[f'conf {i}' for i in range(len(confs))]).T
confs_df.index.names = ['Parameters']
confs_df.to_csv('gridsearch_configurations.csv')

# Destacando apenas os parâmetros que são diferentes entre algumas configurações
confs_df.style.apply(
    lambda x: ['background: lightgreen' if x.name in varying else '' for i in x], 
    axis=1
)

,conf 0,conf 1,conf 2,conf 3,conf 4,conf 5,conf 6,conf 7,conf 8
Parameters,,,,,,,,,
population_size,100.000000,100.000000,100.000000,250.000000,250.000000,250.000000,500.000000,500.000000,500.000000
init_type,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
p_crossover,0.000000,0.200000,0.500000,0.000000,0.200000,0.500000,0.000000,0.200000,0.500000
max_depth_creation,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
tournament_size,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000
zero_depth,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
mutation_step,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
num_random_constants,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
min_random_constant,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000


## GRIDSEARCH

A célula abaixo usa tudo que as células anteriores criaram para rodar um gridsearch. Note que esse processo envolve várias chamadas do terminal e manipulação dos arquivos que o GSGP cria na pasta.

O código é feito de forma que possa ser interrompido, mas não sei se isso pode gerar efeitos colaterais (por conta de abrir arquivos, etc). 

Idealmente, se souber de antemão que o código precisará ser interrompido se exceder o tempo, uma opção é rodar um dataset por vez.

In [14]:
n_folds       = 5
n_runs        = 30
runs_per_fold = n_runs//n_folds

datasets = [
    'airfoil',
    'concrete',
    'energyCooling',
    'energyHeating',
    'GeographicalOriginalofMusic',
    'towerData',
    'tecator',
    'wineRed',
    'wineWhite',
    'yacht',
]    

# ---------------------------
columns = ['dataset','conf','Fold','Rep','RMSE_cv','RMSE_train','RMSE_test']

fname = '../docs/GSGP-resultsregression.csv'
results = {c:[] for c in columns}

if os.path.isfile(fname):
    resultsDF = pd.read_csv(fname)
    results   = resultsDF.to_dict('list')


for ds in datasets:
    print(f'Executando agora para o dataset {ds}')
    for fold in range(n_folds):
        dataset_train_cv      = [f'{adapted_datasets_folder}/{ds}-train-{fold}-train-{i}.txt' for i in range(5)]
        dataset_validation_cv = [f'{adapted_datasets_folder}/{ds}-train-{fold}-validation-{i}.txt' for i in range(5)]
        
        dataset_train         = f'{adapted_datasets_folder}/{ds}-train-{fold}.txt'
        dataset_test          = f'{adapted_datasets_folder}/{ds}-test-{fold}.txt'

        # Verificar se os datasets existem
        if len(glob.glob(dataset_train))==0:
            print(f'Dataset {dataset_train} does not exist.')
            continue
        print(f'Executando para o fold {fold}')

        # Verificar se aquele fold já foi avaliado em alguma repetição, e caso tenha sido
        # pega a configuração utilizada em uma delas (vao ser todas iguais)
        RMSE_cv, conf, conf_id = None, None, None
        if os.path.isfile(fname):
            resultsDF = pd.read_csv(fname)
            results   = resultsDF.to_dict('list')

            if len(resultsDF[
                    (resultsDF['dataset']==ds) &
                    (resultsDF['Fold']==fold)
                ])>0:
                aux_resultsDF = resultsDF[
                    (resultsDF['dataset']==ds) &
                    (resultsDF['Fold']==fold)
                ]
                conf_id = aux_resultsDF['conf'].values[0]
                RMSE_cv = aux_resultsDF['RMSE_cv'].values[0]
                conf = confs[conf_id]

                print(f'Pegando configuração avaliada anteriormente: {RMSE_cv}, {conf_id}')

        # Obtendo melhor configuração para esse treino-teste
        if RMSE_cv == conf == conf_id == None:
            print('Obtendo a melhor configuração utilizando 5-fold cv no treino')
            RMSE_cv, conf, conf_id = gridsearch(dataset_train_cv, dataset_validation_cv, confs)

        print('Começando a testar a melhor configuração obtida')
        for rep in range(runs_per_fold):
            if os.path.isfile(fname):
                resultsDF = pd.read_csv(fname)
                results   = resultsDF.to_dict('list')

                if len(resultsDF[
                    (resultsDF['dataset']==ds) &
                    (resultsDF['Fold']==fold)  &
                    (resultsDF['Rep']==rep)
                ])==1:
                    print(f'already evaluated {ds}-{fold}-{rep}')

                    # Importante notar que ele vê a configuração apenas
                    # pelo ID dela. Se modificar as possíveis configurações, vai gerar bugs.
                    # A ideia aqui é apenas retomar um experimento parado no meio, mas que
                    # não tenha sido modificado durante a pausa.
                    continue

            print(f'evaluating config {conf_id} for {ds}-{fold}-{rep}')
            
            RMSE_train, RMSE_test = run(dataset_train, dataset_test, conf)


            results['dataset'].append(ds)

            # Vamos salvar o número da configuração para ficar mais sucinto
            results['conf'].append(conf_id)
            results['RMSE_cv'].append(RMSE_cv)
            results['RMSE_train'].append(RMSE_train)
            results['RMSE_test'].append(RMSE_test)
            results['Fold'].append(fold)
            results['Rep'].append(rep)

            df = pd.DataFrame(results)
            df.to_csv(fname, index=False)

print('done')

Executando agora para o dataset airfoil
Executando para o fold 0
Obtendo a melhor configuração utilizando 5-fold cv no treino
Testando configuração 0/9
Testando configuração 1/9
Testando configuração 2/9
Testando configuração 3/9
Testando configuração 4/9
Testando configuração 5/9
Testando configuração 6/9
Testando configuração 7/9
Testando configuração 8/9
Começando a testar a melhor configuração obtida
evaluating config 1 for airfoil-0-0
evaluating config 1 for airfoil-0-1
evaluating config 1 for airfoil-0-2
evaluating config 1 for airfoil-0-3
evaluating config 1 for airfoil-0-4
evaluating config 1 for airfoil-0-5
Executando para o fold 1
Obtendo a melhor configuração utilizando 5-fold cv no treino
Testando configuração 0/9
^C
^C


UnboundLocalError: local variable 'child' referenced before assignment

In [15]:
fname = '../docs/GSGP-resultsregression.csv'

resultsDF = pd.read_csv(fname)

pd.set_option('display.max_colwidth', None) #não truncar colunas usando display

# Tá dando crash no número de constantes aleatórias
display(resultsDF)

,dataset,conf,Fold,Rep,RMSE_cv,RMSE_train,RMSE_test
0,airfoil,1,0,0,7.706458,5.90246,5.71633
1,airfoil,1,0,1,7.706458,6.94788,7.32504
2,airfoil,1,0,2,7.706458,7.64804,7.92599
3,airfoil,1,0,3,7.706458,3.99911,4.13304
4,airfoil,1,0,4,7.706458,8.77320,9.20287
5,airfoil,1,0,5,7.706458,16.32530,17.00390


In [16]:
# Obtendo a melhor configuração para cada dataset


# Pegar, para cada dataset-fold-rep, a configuração de menor RMSE_cv
resultsDF_ = resultsDF.loc[resultsDF.groupby(['dataset', 'Fold', 'Rep'])['RMSE_cv'].idxmin()]
resultsDF_ = resultsDF_.set_index(['dataset', 'Fold', 'Rep'])
display(resultsDF_)

# Tirando a média da melhor configuração em cada fold (e descartando 2 primeiras colunas, configuração e cv)
resultsDF_median = resultsDF_.groupby(['dataset']).mean().iloc[:, 2:]
resultsDF_median.columns = ['RMSE_train_mean', 'RMSE_test_mean']
display(resultsDF_median)

# Colocando o desvio padrão e tirando as 2 primeiras colunas (fold e rep, não interessam)
resultsDF_std = resultsDF_.groupby(['dataset']).std().iloc[:, 2:]
resultsDF_std.columns = ['RMSE_train_std', 'RMSE_test_std']
display(resultsDF_std)

# juntando tudo em um só
resultsDF_ = pd.merge(resultsDF_median, resultsDF_std, left_index=True, right_index=True)
display(resultsDF_)

conf   RMSE_cv  RMSE_train  RMSE_test
dataset Fold Rep                                       
airfoil 0    0       1  7.706458     5.90246    5.71633
             1       1  7.706458     6.94788    7.32504
             2       1  7.706458     7.64804    7.92599
             3       1  7.706458     3.99911    4.13304
             4       1  7.706458     8.77320    9.20287
             5       1  7.706458    16.32530   17.00390

,RMSE_train_mean,RMSE_test_mean
dataset,,
airfoil,8.265998,8.551195


,RMSE_train_std,RMSE_test_std
dataset,,
airfoil,4.268994,4.50164


,RMSE_train_mean,RMSE_test_mean,RMSE_train_std,RMSE_test_std
dataset,,,,
airfoil,8.265998,8.551195,4.268994,4.50164
